In [ ]:
# construct graph from nodes and edges
# define starting point
# get all nodes within a certain radius of the starting node. This is equivalent to a maximum driving distance (theoretical straight line from centre to radius and back)
# remove all dead end waypoints
# calculate loop for each waypoint
# score each loop based on parameters



In [2]:
import osmnx as ox
import networkx as nx
import random
import pandas as pd
import folium
from shapely.geometry import LineString
from folium import Map, PolyLine
import sklearn


In [49]:
import geopandas as gpd

# Load saved edges GeoDataFrame
edges = gpd.read_file("C://Users\khong\Documents\Python Projects\Drive\Drive\data/edges_roads_with_curvature.gpkg", layer='edges')
nodes = gpd.read_file("C://Users\khong\Documents\Python Projects\Drive\Drive\data/nodes_roads_with_curvature.gpkg", layer='nodes')


<>:4: SyntaxWarning: invalid escape sequence '\k'
<>:5: SyntaxWarning: invalid escape sequence '\k'
<>:4: SyntaxWarning: invalid escape sequence '\k'
<>:5: SyntaxWarning: invalid escape sequence '\k'
C:\Users\khong\AppData\Local\Temp\ipykernel_13664\3870012126.py:4: SyntaxWarning: invalid escape sequence '\k'
  edges = gpd.read_file("C://Users\khong\Documents\Python Projects\Drive\Drive\data/edges_roads_with_curvature.gpkg", layer='edges')
C:\Users\khong\AppData\Local\Temp\ipykernel_13664\3870012126.py:5: SyntaxWarning: invalid escape sequence '\k'
  nodes = gpd.read_file("C://Users\khong\Documents\Python Projects\Drive\Drive\data/nodes_roads_with_curvature.gpkg", layer='nodes')


In [50]:
edges.set_index(['u', 'v', 'key'], inplace=True)
nodes.set_index(['osmid'], inplace=True)

In [51]:
# Reconstruct the graph
G = ox.graph_from_gdfs(nodes, edges)

In [6]:
start_lat = 53.1701   # Parsley Hay Car Park lat
start_lon = -1.7816  # Parsley Hay Car Park lon

start_node = ox.nearest_nodes(G, start_lon, start_lat)


In [7]:
import osmnx.distance

def get_random_waypoints(G, start_lat, start_lon, num_waypoints=3, radius=10000):
    start_node = ox.nearest_nodes(G, start_lon, start_lat)
    nodes = list(G.nodes(data=True))

    # Filter nodes within radius
    candidate_nodes = [
        (n, data) for n, data in nodes
        if osmnx.distance.great_circle(start_lat, start_lon, data['y'], data['x']) < radius
    ]

    if len(candidate_nodes) < num_waypoints:
        raise ValueError("Not enough nearby nodes to sample waypoints.")

    sampled = random.sample(candidate_nodes, num_waypoints)
    return [(data['y'], data['x']) for _, data in sampled]


# Generate waypoints
waypoints = get_random_waypoints(G, start_lat, start_lon, num_waypoints=10)
# --- Create Folium map ---
m = folium.Map(location=[start_lat, start_lon], zoom_start=11)

# Plot start
folium.Marker([start_lat, start_lon], popup="Start", icon=folium.Icon(color='green')).add_to(m)

# Plot waypoints
for idx, (lat, lon) in enumerate(waypoints, start=1):
    folium.Marker([lat, lon], popup=f"Waypoint {idx}", icon=folium.Icon(color='blue')).add_to(m)

radius = 10000
folium.Circle(
    location=[start_lat, start_lon],
    radius=radius,
    color="black",
    weight=1,
    opacity=1,
    fill=False,  # gets overridden by fill_color
).add_to(m)

# Show map
m

In [72]:
from networkx import shortest_path
import networkx as nx

def generate_loop_paths(G, start_node, waypoint_nodes, weight="length"):
    loop_paths = []

    waypoint_nodes = [ox.nearest_nodes(G, lon, lat) for lat, lon in waypoint_nodes]

    for wp in waypoint_nodes:
        try:
            # Outbound path
            out_path = nx.shortest_path(G, source=start_node, target=wp, weight=weight)

            # Remove outbound edges temporarily to discourage backtracking
            G_temp = G.copy()
            out_edges = list(zip(out_path[:-1], out_path[1:]))
            G_temp.remove_edges_from(out_edges)

            # Inbound path
            try:
                in_path = nx.shortest_path(G_temp, source=wp, target=start_node, weight=weight)
            except nx.NetworkXNoPath:
                # fallback to original graph if no return path
                in_path = nx.shortest_path(G, source=wp, target=start_node, weight=weight)

            # Combine into loop and store
            full_loop = out_path + in_path[1:]
            loop_paths.append(full_loop)

        except nx.NetworkXNoPath:
            print(f"No path found for waypoint {wp}")
            continue

    return loop_paths

loop_paths = generate_loop_paths(G, start_node, waypoints)

In [15]:
import networkx as nx
from copy import deepcopy

def get_paths_avoiding_outbound(G, start_node, waypoints, weight='length'):
    paths = []
    waypoints = [ox.nearest_nodes(G, lon, lat) for lat, lon in waypoints]

    for wp in waypoints:
        try:
            # Outbound path
            path_out = nx.shortest_path(G, source=start_node, target=wp, weight=weight)

            # Collect all used edges in (u, v, key) format
            outbound_edges = []
            for u, v in zip(path_out[:-1], path_out[1:]):
                for key in G[u][v]:  # all parallel edges
                    outbound_edges.append((u, v, key))
                    outbound_edges.append((v, u, key))

            # Create a copy of the graph and remove outbound edges
            G_return = deepcopy(G)
            G_return.remove_edges_from(outbound_edges)

            # In case removal breaks connectivity, wrap in try
            path_back = nx.shortest_path(G_return, source=wp, target=start_node, weight=weight)

            # Combine paths
            full_path = path_out + path_back[1:]  # avoid repeating the waypoint
            paths.append(full_path)

        except (nx.NetworkXNoPath, nx.NodeNotFound):
            print(f"No valid path for waypoint: {wp}")
            continue

    return paths

loop_paths=get_paths_avoiding_outbound(G, start_node, waypoints)

No valid path for waypoint: 706134889
No valid path for waypoint: 706507959
No valid path for waypoint: 1988435296
No valid path for waypoint: 771340148


In [9]:
# --- Create Folium map ---
m = folium.Map(location=[start_lat, start_lon], zoom_start=14)

# Plot start
folium.Marker([start_lat, start_lon], popup="Start", icon=folium.Icon(color='green')).add_to(m)


from itertools import cycle

# Color cycle for the paths
colors = cycle(['red', 'blue', 'green', 'purple', 'orange'])

for path in loop_paths:
    coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in path]
    folium.PolyLine(coords, color=next(colors), weight=5, opacity=0.7).add_to(m)


# Plot waypoints
for idx, (lat, lon) in enumerate(waypoints, start=1):
    folium.Marker([lat, lon], popup=f"Waypoint {idx}", icon=folium.Icon(color='blue')).add_to(m)

# Show map
m

In [21]:
def score_path_by_curvature(G, path):
    total_curvature = 0
    total_length = 0

    for u, v in zip(path[:-1], path[1:]):
        # Get all parallel edges between u and v (MultiDiGraph may have >1)
        edges = G.get_edge_data(u, v)
        if edges:
            # Select the shortest edge (or first) if multiple exist
            edge_data = min(edges.values(), key=lambda d: d.get('length', float('inf')))
            curvature = edge_data.get('curvature', 0)
            length = edge_data.get('length', 1)
            total_curvature += curvature
            total_length += length

    # Normalize: curvature per km (optional)
    curvature_score = total_curvature # / (total_length / 1000) if total_length else 0
    return int(curvature_score), int(total_length)


In [88]:
selected_path = 3
path = loop_paths[selected_path]
print(score_path_by_curvature(G, loop_paths[selected_path]))

path_edges = []

for u, v in zip(path[:-1], path[1:]):
    # Get all parallel edges between u and v (MultiDiGraph may have >1)
    edge = G.get_edge_data(u, v)
    
    if edge:
        # Select the shortest edge (or first) if multiple exist
        edge_data = min(edge.values(), key=lambda d: d.get('length', float('inf')))
        path_edges.append(edge_data)
        curvature = edge_data.get('curvature', 0)

print(path_edges)

(21776, 28362)
[{'osmid': '862832873', 'highway': 'trunk', 'name': 'Buxton Road', 'oneway': False, 'reversed': 'True', 'length': 62.84150049899899, 'lanes': '2', 'maxspeed': '50 mph', 'ref': 'A515', 'curvature': 0.0, 'geometry': <LINESTRING (-1.78 53.17, -1.78 53.169)>}, {'osmid': '22689817', 'highway': 'tertiary', 'oneway': False, 'reversed': 'True', 'length': 1114.6650312024494, 'curvature': 476.0, 'geometry': <LINESTRING (-1.78 53.169, -1.78 53.169, -1.78 53.169, -1.781 53.169, -1.782...>}, {'osmid': '22689815', 'highway': 'unclassified', 'oneway': False, 'reversed': 'False', 'length': 753.5872059151086, 'curvature': 652.0, 'geometry': <LINESTRING (-1.796 53.17, -1.797 53.17, -1.798 53.169, -1.798 53.169, -1.79...>}, {'osmid': '[33573833, 35302631]', 'highway': "['unclassified', 'residential']", 'name': "['Hyde Lane', 'Hide Lane']", 'oneway': False, 'reversed': 'False', 'length': 2778.5344650904767, 'curvature': 1141.0, 'geometry': <LINESTRING (-1.798 53.164, -1.798 53.164, -1.798 5

In [89]:
import matplotlib.pyplot as plt

def get_color(val, vmin, vmax):
    norm = plt.Normalize(vmin=vmin, vmax=vmax)
    cmap = plt.get_cmap('viridis')
    rgba = cmap(norm(val))
    return f'rgba({int(rgba[0]*255)}, {int(rgba[1]*255)}, {int(rgba[2]*255)}, {rgba[3]})'

# --- Create Folium map ---
m = folium.Map(location=[start_lat, start_lon], zoom_start=11)

# Plot start
folium.Marker([start_lat, start_lon], popup="Start", icon=folium.Icon(color='green')).add_to(m)

coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in path]

#  folium.PolyLine(coords, color='blue', weight=5, opacity=0.7).add_to(m)

# Add edges to the map
for row in path_edges:
    if isinstance(row['geometry'], list):
        # Skip any weird geometry
        continue
    folium.PolyLine(locations=[(lat, lon) for lon, lat in row['geometry'].coords],
                    color=get_color(row['curvature'], edges['curvature'].min(), edges['curvature'].max()),
                    weight=5).add_to(m)

# Show map
m